## Structured Outputs

This Jupyter notebook shows examples of structured outputs with the GWDG SAIA API.

[Source](https://docs.vllm.ai/en/latest/features/structured_outputs.html)

#### Prequesites
1. To install the required packages remove the comment character before the next line
2. Add your API Key to a .env file in the root directory (see .env.example file). 
3. Load the API key from the .env file (see the lines below)

In [1]:
# !pip install openai python-dotenv

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

#### Configure Client

In [3]:
from openai import OpenAI

api_key = os.getenv("API_KEY")
base_url = "https://chat-ai.academiccloud.de/v1"

client = OpenAI(
    base_url=base_url,
    api_key=api_key,
    )

#### Get Model List

In [4]:
models = client.models.list()

for i, model in enumerate(models.model_dump()["data"]):
    print(f"{i}: {model['id']}, {model['input']} -> {model['output']}")

0: meta-llama-3.1-8b-instruct, ['text'] -> ['text']
1: openai-gpt-oss-120b, ['text'] -> ['text']
2: gemma-3-27b-it, ['text', 'image'] -> ['text']
3: qwen3-32b, ['text'] -> ['text', 'thought']
4: qwen3-235b-a22b, ['text'] -> ['text', 'thought']
5: llama-3.3-70b-instruct, ['text'] -> ['text']
6: qwen2.5-vl-72b-instruct, ['text', 'image', 'video'] -> ['text']
7: medgemma-27b-it, ['text', 'image'] -> ['text']
8: qwq-32b, ['text'] -> ['text', 'thought']
9: deepseek-r1, ['text'] -> ['text', 'thought']
10: deepseek-r1-distill-llama-70b, ['text'] -> ['text', 'thought']
11: mistral-large-instruct, ['text'] -> ['text']
12: qwen2.5-coder-32b-instruct, ['text'] -> ['text']
13: internvl2.5-8b, ['text', 'image'] -> ['text']
14: teuken-7b-instruct-research, ['text'] -> ['text']
15: codestral-22b, ['text'] -> ['text']
16: llama-3.1-sauerkrautlm-70b-instruct, ['text', 'arcana'] -> ['text']
17: meta-llama-3.1-8b-rag, ['text', 'arcana'] -> ['text']


#### Structured Output from `guided_choice`

In [5]:
model = client.models.list().data[0].id

completion = client.chat.completions.create( 
    model=model, 
    messages=[
        {
            "role": "user", 
            "content": "Classify this sentiment: Chat AI is wonderful!"
            } 
    ],
    extra_body={"guided_choice": ["positive", "negative"]}, 
    ) 

print(completion.choices[0].message.content)


positive


#### Structured Output with `guided_regex`

In [6]:
model = client.models.list().data[5].id

completion = client.chat.completions.create( 
    model=model, 
    messages=[ 
        {
            "role": "user", 
            "content": "Generate an example email address for Alan Turing, who works in Enigma. End in .com and new line. Example result: alan.turing@enigma.com\n",
            } 
    ], 
    extra_body={"guided_regex": r"\w+@\w+\.com\n", "stop": ["\n"]}, 
) 

print(completion.choices[0].message.content)


alan_turing@enigma.com


#### Structured Output as `json_schema`

In [7]:
from pydantic import BaseModel

model = client.models.list().data[2].id

class People(BaseModel):
    name: str
    age: int

completion = client.chat.completions.create( 
    model=model, 
    messages=[
        {
            "role": "user", 
            "content": "Generate a JSON with the name and age of one random person.",
        }
    ], 
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "people",
            "schema": People.model_json_schema()
        },
    }, 
) 

print(completion.choices[0].message.content)

{
  "name": "Eleanor Vance",
  "age": 32
}


#### Optional: With Reasoning Outputs (if Reasoning Models)

In [8]:
print("reasoning_content: ", completion.choices[0].message.reasoning_content)


reasoning_content:  None


#### Optional: With JSON Validation

In [9]:
try:
    People.model_validate_json(completion.choices[0].message.content)
    print(completion.choices[0].message.content)
except Exception as e:
    print(f"⚠️ JSON error: {e}")

{
  "name": "Eleanor Vance",
  "age": 32
}
